# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,32.41,88,42,9.10,GS,1697532205
1,1,ushuaia,-54.8000,-68.3000,44.26,70,75,13.80,AR,1697532080
2,2,kuala terengganu,5.3302,103.1408,85.95,79,20,7.00,MY,1697532309
3,3,port mathurin,-19.6833,63.4167,76.66,81,2,11.99,MU,1697532310
4,4,nizhnesortymskiy,62.4397,71.7603,50.45,88,98,11.12,RU,1697532311


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

city_data_df = city_data_df.sort_values("City",ascending=False)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
                                        "Lng",
                                        "Lat",
                                        geo = True,
                                        tiles = "OSM",
                                        frame_width = 800,
                                        frame_height = 600,
                                        size = "Humidity",
                                        scale = 0.7, 
                                        color = "City",
                                        title = "City Weather Map"
   
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cleand_city_data = city_data_df[(city_data_df["Wind Speed"] <= 15) & (city_data_df["Cloudiness"] <= 100) & (city_data_df["Max Temp"] <= 100) & (city_data_df["Max Temp"] >= 19) & (city_data_df["Country"] == "AU")]

# Drop any rows with null values
cleand_city_data.dropna() 
# Display sample data
cleand_city_data.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
409,409,wynyard,-41.0000,145.7167,55.27,62,2,9.28,AU,1697532843
469,469,warrnambool,-38.3833,142.4833,51.21,72,3,9.40,AU,1697532921
199,199,port lincoln,-34.7333,135.8667,56.73,61,1,9.86,AU,1697532571
271,271,port augusta,-32.5000,137.7667,64.45,45,0,12.48,AU,1697532664
140,140,parkes,-33.1333,148.1833,58.87,37,0,8.43,AU,1697532490
276,276,new norfolk,-42.7826,147.0587,54.91,79,12,3.22,AU,1697532670
233,233,mount isa,-20.7333,139.5000,87.57,12,0,9.22,AU,1697532615
31,31,mount gambier,-37.8333,140.7667,51.08,76,0,6.93,AU,1697532343
222,222,howard springs,-12.4970,131.0470,89.51,39,46,7.45,AU,1697532601
292,292,geraldton,-28.7667,114.6000,72.01,73,40,13.80,AU,1697532689


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleand_city_data.loc[:, ["City", "Country", "Lat", "Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
409,wynyard,AU,-41.0000,145.7167,62,
469,warrnambool,AU,-38.3833,142.4833,72,
199,port lincoln,AU,-34.7333,135.8667,61,
271,port augusta,AU,-32.5000,137.7667,45,
140,parkes,AU,-33.1333,148.1833,37,
276,new norfolk,AU,-42.7826,147.0587,79,
233,mount isa,AU,-20.7333,139.5000,12,
31,mount gambier,AU,-37.8333,140.7667,76,
222,howard springs,AU,-12.4970,131.0470,39,
292,geraldton,AU,-28.7667,114.6000,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {
     "categories":"accommodation.hotel",
     "apiKey": geoapify_key,
     "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame    
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]   
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wynyard - nearest hotel: Wynyard Hotel
warrnambool - nearest hotel: Central Court
port lincoln - nearest hotel: Boston Hotel
port augusta - nearest hotel: Hotel Commonwealth
parkes - nearest hotel: Commercial Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
mount isa - nearest hotel: Ibis Styles
mount gambier - nearest hotel: The Old Mount Gambier Gaol
howard springs - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
darwin - nearest hotel: Mantra Pantanas Darwin
cooma - nearest hotel: Royal Hotel
codrington - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
blackmans bay - nearest hotel: Villa Howden
aldinga beach - nearest hotel: The Beach Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
409,wynyard,AU,-41.0000,145.7167,62,Wynyard Hotel
469,warrnambool,AU,-38.3833,142.4833,72,Central Court
199,port lincoln,AU,-34.7333,135.8667,61,Boston Hotel
271,port augusta,AU,-32.5000,137.7667,45,Hotel Commonwealth
140,parkes,AU,-33.1333,148.1833,37,Commercial Hotel
276,new norfolk,AU,-42.7826,147.0587,79,Woodbridge on the Derwent
233,mount isa,AU,-20.7333,139.5000,12,Ibis Styles
31,mount gambier,AU,-37.8333,140.7667,76,The Old Mount Gambier Gaol
222,howard springs,AU,-12.4970,131.0470,39,No hotel found
292,geraldton,AU,-28.7667,114.6000,73,Ocean Centre Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

hotel_df = hotel_df.sort_values("City",ascending=False)

# Configure the map plot
map_plot_2= hotel_df.hvplot.points( "Lng",
                                    "Lat",                   
                                    geo = True,
                                    tiles = "EsriImagery",
                                    hover_cols=['Hotel Name', 'Country'],
                                    frame_width = 800,
                                    frame_height = 600,
                                    size = "Humidity",
                                    cmap='RdYlGn',
                                    scale = 0.7, 
                                    color = "City", 
                                    title='Hotel Map'
                                  )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)